In [16]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
import numpy as np
import matplotlib.pyplot as plt

In [17]:
base_model = VGG19(weights='imagenet', include_top=False)

In [18]:
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_conv4').output)

In [19]:
img_path = 'image.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_data = image.img_to_array(img)
img_data = np.expand_dims(img_data, axis=0)
img_data = preprocess_input(img_data)  # Preprocess input image for VGG19

In [20]:
features = model.predict(img_data)
print("Feature map shape:", features.shape)

1/1 [==============================] - 0s 338ms/step
Feature map shape: (1, 14, 14, 512)


In [24]:
# Create models for each layer to extract features
layer_outputs = [layer.output for layer in base_model.layers if 'conv4' in layer.name]
layer_names = [layer.name for layer in base_model.layers if 'conv4' in layer.name]
feature_model = Model(inputs=base_model.input, outputs=layer_outputs)

In [25]:
# Extract feature maps for all layers
features = feature_model.predict(img_data)

1/1 [==============================] - 0s 414ms/step


In [26]:
for layer_name, feature_map in zip(layer_names, features):
    # Get the number of feature maps for the layer
    num_feature_maps = feature_map.shape[-1]
    
    # Loop over each feature map
    for i in range(num_feature_maps):
        # Extract the ith feature map
        feature = feature_map[0, :, :, i]`
        
        # Normalize the feature map for visualization
        feature -= feature.mean()
        feature /= (feature.std() + 1e-5)  # Avoid division by zero
        feature *= 64
        feature += 128
        feature = np.clip(feature, 0, 255).astype('uint8')
        
        # Plot and save the feature map as an image
        plt.imshow(feature, cmap='viridis')  # Use a colormap for better visualization
        plt.axis('off')
        plt.savefig(f'{layer_name}_feature_{i}.png', bbox_inches='tight')
        plt.close()

print("Feature maps saved successfully.")

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>